In [1]:
import pandas as pd
import numpy as np
import lightfm_wrapper

%load_ext autoreload
%autoreload 2

c:\users\david\desktop\sandbox\pyrecs\env\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [15]:
df = pd.DataFrame({'users':['user1','user1','user1','user1','user2'], 
                   'items':['item1','item1','item3','item1','item2']})
df

,users,items
0,user1,item1
1,user1,item1
2,user1,item3
3,user1,item1
4,user2,item2


In [16]:
interactions_type = 'counts'
lfm = lightfm_wrapper.LightFM(rows_col='users', columns_col='items', interactions_type=interactions_type)

In [17]:
lfm._make_interactions_matrix(df.copy())

In [39]:
interactions_matrix = lfm.interactions_matrix.todense()
interactions_matrix

matrix([[3, 1, 0],
        [0, 0, 1],
        [0, 0, 0]], dtype=int64)

In [31]:
interactions_matrix2 = interactions_matrix.copy()

In [32]:
interactions_matrix2[interactions_matrix2 >= 1] = 1

In [34]:
interactions_matrix2

matrix([[1, 1, 0],
        [0, 0, 1],
        [0, 0, 0]], dtype=int64)

In [33]:
interactions_matrix

matrix([[3, 1, 0],
        [0, 0, 1],
        [0, 0, 0]], dtype=int64)

In [19]:
if interactions_type == 'ones':
    df.drop_duplicates(inplace=True)

In [20]:
# Check that rows have the right number of interactions
row_counts = df.groupby('users').size().reset_index().rename(columns={0:'count'})
row_counts['user_inds'] = row_counts['users'].map(lfm.rows2ind)
rowind2count = dict(zip(row_counts['user_inds'], row_counts['count']))
for row, count in rowind2count.items():
    assert interactions_matrix[row].sum() == count

In [21]:
# Check that columns have the right number of interactions
column_counts = df.groupby('items').size().reset_index().rename(columns={0:'count'})
column_counts['item_inds'] = column_counts['items'].map(lfm.columns2ind)
columnind2count = dict(zip(column_counts['item_inds'], column_counts['count']))
for column, count in columnind2count.items():
    assert interactions_matrix[:,column].sum() == count

In [22]:
# Check that there are the right number of empty interactions
if df['users'].nunique() == df['items'].nunique():
    smaller_dim = None
    assert len(rowind2count) == len(columnind2count)
    assert len(rowind2count) == interactions_matrix.shape[0]
    assert len(columnind2count) == interactions_matrix.shape[1]
elif df['users'].nunique() < df['items'].nunique():
    smaller_dim = 1
else:
    smaller_dim = 0
if smaller_dim is not None:
    num_zero_sums = len(np.where(np.array(interactions_matrix.sum(axis=1)).flatten()==0)[0])
    assert num_zero_sums == len(columnind2count)-len(rowind2count)